The idea of the visualization and the base code was taken from https://github.com/AIML-IfI/openset-imagenet-comparison. It was developed **by Palechor, Andres and Bhoumik, Annesha and Günther, Manuel** for the paper "**Large-Scale Open-Set Classification Protocols for ImageNet**" (Winter Conference on Applications of Computer Vision (WACV)) in 2023. You find the citation in the readme file.

We have adapted it to fit our needs to visualize many different methods.

In [ ]:
import argparse
import multiprocessing
import collections
import subprocess
import pathlib
import os, sys
import torch
import numpy

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

from collections import defaultdict

from matplotlib import pyplot, cm, colors
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MaxNLocator, LogLocator

from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import ticker
from matplotlib.lines import Line2D
from matplotlib.ticker import LogLocator, NullFormatter
from matplotlib import colors
import matplotlib.cm as cm

import yaml

THRESHOLDS = {1e-4: "$10^{-4}$",
              1e-3: "$10^{-3}$",
              1e-2: "$10^{-2}$",
              1e-1: "$10^{-1}$",
              0.2: "$0.2$",
              1: "$1$",
}



# get distinguishable colors
import matplotlib.cm
colors = matplotlib.cm.tab10(range(10))

COLORS = {
    "threshold": colors[0],
    "openmax": colors[8],
    "proser": colors[2],
    "evm": colors[3],
    "maxlogits": colors[5]
}

STYLES = {
    "entropic": "dashed",
    "softmax": "solid",
    "garbage": "dotted",
    "p1": "dashed",
    "p2": "dotted",
    "p3": "solid"
}

NAMES = {
    "threshold": "Threshold",
    "openmax": "OpenMax",
    "proser": "PROSER",
    "evm": "EVM",
    "maxlogits": "MaxLogits",
    "entropic": "EOS",
    "softmax": "Softmax",
    "garbage": "Garbage",
    "p1": "P_1",
    "p2": "P_2",
    "p3": "P_3",
    1: "$P_1$",
    2: "$P_2$",
    3: "$P_3$"
}

from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import ticker
from matplotlib.lines import Line2D
from matplotlib.ticker import LogLocator, NullFormatter
from matplotlib import colors
import matplotlib.cm as cm

import yaml

In [ ]:
#function to load data from npz-file
def load_scores(use_best, algorithm, loss, protocol ,method, eval, experiment_number):
    suffix = "best" if use_best else "curr"
    scores = defaultdict(lambda: defaultdict(dict))
    ground_truths = {}
    scr = "scores"

    score_file = f"{loss}_{algorithm}_{eval}_arr_{suffix}_{str(experiment_number)}.npz"

    if(os.path.exists(score_file)):
        results = numpy.load(score_file)
        scores[protocol][loss][algorithm] = results[scr]
        if(protocol not in ground_truths):
            ground_truths[protocol] = results["gt"].astype(int)
        else:
            assert numpy.all(results["gt"] == ground_truths[protocol])

    else:
      print("did not find the data")

    return scores, ground_truths

In [ ]:
#function for histogram figure
def get_histogram(scores,
                  gt,
                  bins=100,
                  log_space=False,
                  geomspace_limits=(1, 1e2)):
    """Calculates histograms of scores"""
    known = gt >= 0
    unknown = gt == -2
    negative = gt == -1

    knowns = scores[known, gt[known]]
    unknowns = np.amax(scores[unknown], axis=1)
    negatives = np.amax(scores[negative], axis=1)

    if log_space:
        lower, upper = geomspace_limits
        bins = np.geomspace(lower, upper, num=bins)
#    else:
#        bins = np.linspace(0, 1, num=bins+1)
    histograms = {}
    histograms["known"] = np.histogram(knowns, bins=bins)
    histograms["unknown"] = np.histogram(unknowns, bins=bins)
    histograms["negative"] = np.histogram(negatives, bins=bins)
    return histograms


In [ ]:
#version for displaying the score distribution of 5 methods
def plot_score_distributions( scores, ground_truths,
                             scores1, ground_truths1,
                              scores2, ground_truths2,
                              scores3, ground_truths3,
                              scores4, ground_truths4,
                              scores5, ground_truths5 ,
                              scores6, ground_truths6 ,
                              scores7, ground_truths7 ,
                              scores8, ground_truths8,
                              scores9, ground_truths9 ,
                              scores10, ground_truths10 ,
                              scores11, ground_truths11 ,
                              scores12, ground_truths12,
                              scores13, ground_truths13 ,
                              scores14, ground_truths14,


                              protocols,protocols1,protocols2,protocols3,protocols4,protocols5,protocols6,protocols7,protocols8, protocols9,protocols10,protocols11,protocols12,protocols13, protocols14,
                              loss, algorithms,  pdf,

                              title, title1, title2, title3, title4, title5,title6, title7, title8,title9, title10, title11,title12, title13, title14):
    #define parameters
    font_size = 10
    bins = 120
    P = 1
    L = 1
    A = len(algorithms)

    # Manual colors
    edge_unknown = colors.to_rgba('indianred', 1)
    fill_unknown = colors.to_rgba('firebrick', 0.04)
    edge_known = colors.to_rgba('tab:blue', 1)
    fill_known = colors.to_rgba('tab:blue', 0.04)
    edge_negative = colors.to_rgba('tab:green', 1)
    fill_negative = colors.to_rgba('tab:green', 0.04)

    #define the figure
    fig  = pyplot.figure(figsize = (20,12))
    gs = fig.add_gridspec(3,5,hspace = 0.2, wspace = 0.5)
    ax = gs.subplots(sharex = True, sharey = 'row')
    ax = ax.flat

    #plot all the histograms for each protocol and method and set the title accordingly
    if scores[protocols][loss][algorithms] is not None:
      histograms = get_histogram(scores[protocols][loss][algorithms], ground_truths[protocols], bins = bins)
      ax[0].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[0].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[0].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[0].set_title(title,size = font_size)


    if scores1[protocols1][loss][algorithms] is not None:
      histograms = get_histogram(scores1[protocols1][loss][algorithms], ground_truths1[protocols1], bins = bins)
      ax[1].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[1].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[1].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[1].set_title(title1,size = font_size)


    if scores2[protocols2][loss][algorithms] is not None:
      histograms = get_histogram(scores2[protocols2][loss][algorithms], ground_truths2[protocols2], bins = bins)
      ax[2].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[2].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[2].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[2].set_title(title2,size = font_size)


    if scores3[protocols3][loss][algorithms] is not None:
      histograms = get_histogram(scores3[protocols3][loss][algorithms], ground_truths3[protocols3], bins = bins)
      ax[3].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[3].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[3].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[3].set_title(title3,size = font_size)


    if scores4[protocols4][loss][algorithms] is not None:
      histograms = get_histogram(scores4[protocols4][loss][algorithms], ground_truths4[protocols4], bins = bins)
      ax[4].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[4].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[4].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[4].set_title(title4,size = font_size)

    if scores5[protocols5][loss][algorithms] is not None:
      histograms = get_histogram(scores5[protocols5][loss][algorithms], ground_truths5[protocols5], bins = bins)
      ax[5].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[5].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[5].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[5].set_title(title5,size = font_size)


    if scores6[protocols6][loss][algorithms] is not None:
      histograms = get_histogram(scores6[protocols6][loss][algorithms], ground_truths6[protocols6], bins = bins)
      ax[6].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[6].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[6].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[6].set_title(title6,size = font_size)



    if scores7[protocols7][loss][algorithms] is not None:
      histograms = get_histogram(scores7[protocols7][loss][algorithms], ground_truths7[protocols7], bins = bins)
      ax[7].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[7].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[7].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[7].set_title(title7,size = font_size)


    if scores8[protocols8][loss][algorithms] is not None:
      histograms = get_histogram(scores8[protocols8][loss][algorithms], ground_truths8[protocols8], bins = bins)
      ax[8].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[8].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[8].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[8].set_title(title8,size = font_size)

    if scores9[protocols9][loss][algorithms] is not None:
      histograms = get_histogram(scores9[protocols9][loss][algorithms], ground_truths9[protocols9], bins = bins)
      ax[9].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[9].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[9].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[9].set_title(title9,size = font_size)

    if scores10[protocols10][loss][algorithms] is not None:
      histograms = get_histogram(scores10[protocols10][loss][algorithms], ground_truths10[protocols10], bins = bins)
      ax[10].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[10].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[10].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[10].set_title(title10,size = font_size)

    if scores11[protocols11][loss][algorithms] is not None:
      histograms = get_histogram(scores11[protocols11][loss][algorithms], ground_truths11[protocols11], bins = bins)
      ax[11].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[11].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[11].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[11].set_title(title11,size = font_size)

    if scores12[protocols12][loss][algorithms] is not None:
      histograms = get_histogram(scores12[protocols12][loss][algorithms], ground_truths12[protocols12], bins = bins)
      ax[12].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[12].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[12].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[12].set_title(title12,size = font_size)

    if scores13[protocols13][loss][algorithms] is not None:
      histograms = get_histogram(scores13[protocols13][loss][algorithms], ground_truths13[protocols13], bins = bins)
      ax[13].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[13].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[13].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[13].set_title(title13,size = font_size)

    if scores14[protocols14][loss][algorithms] is not None:
      histograms = get_histogram(scores14[protocols14][loss][algorithms], ground_truths14[protocols14], bins = bins)
      ax[14].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[14].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[14].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[14].set_title(title14,size = font_size)

    from matplotlib.lines import Line2D
    legend_elements = [Line2D([None], [None], color=edge_known),
                        Line2D([None], [None], color=edge_negative),
                        Line2D([None], [None], color=edge_unknown)]
    legend_labels = ["Known", "Negative", "Unknown"]
    fig.legend(handles=legend_elements, labels=legend_labels,  ncol=3, loc = "lower center", bbox_to_anchor = (0.51,0.04))

    return fig

In [ ]:
#visualizing and using the score distribution function
use_best = False
algorithm = 'threshold'
protocol =protocol1 = protocol2 =protocol3 = protocol4 = protocol5= protocol6 = protocol7 = protocol8 =  '2'
method = 'noise'
loss ='entropic'

#default parameters
benchmark_score = '-'
benchmark_ground_truth = ground_truths1 = ground_truths = ground_truths2 = 0
scores = '-'
scores1 = '-'
scores2 = '-'

#column 1
protocol = '1'
protocol5 = '2'
protocol10 ='3'
scores, ground_truths = load_scores(use_best, algorithm, loss, protocol ,method, 'test','benchmark_p1') #red
scores5, ground_truths5 = load_scores(use_best, algorithm, loss, protocol5 ,method, 'test','benchmark_p2') #red
scores10, ground_truths10 = load_scores(use_best, algorithm, loss, protocol10 ,method, 'test','benchmark_p3') #red
title = 'P1 Benchmark'
title5 = 'P2 Benchmark'
title10 = 'P3 Benchmark'

#column 2
protocol1 = '1'
protocol6 = '2'
protocol11 ='3'
scores1, ground_truths1 = load_scores(use_best, algorithm, loss, protocol1 ,method, 'test','p1')
scores6, ground_truths6 = load_scores(use_best, algorithm, loss, protocol6 ,method, 'test','p2')
scores11, ground_truths11 = load_scores(use_best, algorithm, loss, protocol11 ,method, 'test','p3')
title1 = 'P1 '
title6 = 'P2 '
title11 = 'P3 '


#column 3
protocol2 = '1'
protocol7 = '2'
protocol12 ='3'
scores2, ground_truths2 = load_scores(use_best, algorithm, loss, protocol2 ,method, 'test','p1')
scores7, ground_truths7 = load_scores(use_best, algorithm, loss, protocol7 ,method, 'test','p2')
scores12, ground_truths12 = load_scores(use_best, algorithm, loss, protocol12 ,method, 'test','p3')
title2 = 'P1 Mixing-Technique at $S_2$ + Logit-Filter'
title7 = 'P2 Mixing-Technique at $S_2$ + Logit-Filter'
title12 = 'P3 Mixing-Technique at $S_2$ + Logit-Filter'

#column 4
protocol3 = '1'
protocol8 = '2'
protocol13 ='3'
scores3, ground_truths3 = load_scores(use_best, algorithm, loss, protocol3 ,method, 'test', 'noise_stage2_p1')
scores8, ground_truths8 = load_scores(use_best, algorithm, loss, protocol5 ,method, 'test', 'noise_stage2_p2')
scores13, ground_truths13 = load_scores(use_best, algorithm, loss, protocol13 ,method, 'test', 'noise_stage2_p3')
title3 = 'P1 Noise Injection Method at $S_2$'
title8 = 'P2 Noise Injection Method at $S_2$'
title13 = 'P3 Noise Injection Method at $S_2$'


#column 5
protocol4 = '1'
protocol9 = '2'
protocol14 ='3'
scores4, ground_truths4 = load_scores(use_best, algorithm, loss, protocol4 ,method, 'test','p1')
scores9, ground_truths9 = load_scores(use_best, algorithm, loss, protocol9,method, 'test','p2')
scores14, ground_truths14 = load_scores(use_best, algorithm, loss, protocol14 ,method, 'test','p3')
title4 = 'P1'
title9 ='P2'
title14 = 'P3'


name = 'Comparison_Stage3.1_curr_all_ScoreDistribution'

output_file_directory = f"{name}.pdf"


pdf = PdfPages(output_file_directory)


fig1 = plot_score_distributions( scores,ground_truths,
                                scores1, ground_truths1 ,
                                 scores2, ground_truths2,
                                 scores3, ground_truths3,
                                 scores4, ground_truths4,
                                 scores5, ground_truths5,
                                 scores6, ground_truths6,
                                 scores7, ground_truths7,
                                 scores8, ground_truths8,
                                 scores9, ground_truths9,
                                 scores10, ground_truths10,
                                 scores11, ground_truths11,
                                 scores12, ground_truths12,
                                 scores13, ground_truths13,
                                 scores14, ground_truths14,


                                 protocol,protocol1,protocol2, protocol3, protocol4, protocol5, protocol6, protocol7, protocol8,protocol9, protocol10, protocol11, protocol12, protocol13, protocol14,
                                 loss, algorithm,  pdf,

                                 title, title1, title2,title3, title4, title5,title6, title7, title8,title9, title10, title11,title12, title13, title14)



pyplot.show()
pdf.savefig(fig1, bbox_inches = 'tight')

pdf.close()

In [ ]:
#version for 6 methods
def plot_score_distributions6(scores,ground_truths,
                                scores1, ground_truths1 ,
                                 scores2, ground_truths2,
                                 scores3, ground_truths3,
                                 scores4, ground_truths4,
                                 scores5, ground_truths5,
                                 scores6, ground_truths6,
                                 scores7, ground_truths7,
                                 scores8, ground_truths8,
                                 scores9, ground_truths9,
                                 scores10, ground_truths10,
                                 scores11, ground_truths11,
                                 scores12, ground_truths12,
                                 scores13, ground_truths13,
                                 scores14, ground_truths14,
                                scores15, ground_truths15,
                                 scores16, ground_truths16,
                                 scores17, ground_truths17,



                                 protocols,protocols1,protocols2, protocols3, protocols4, protocols5, protocols6, protocols7, protocols8,protocols9, protocols10, protocols11, protocols12, protocols13, protocols14, protocols15, protocols16, protocols17,
                                 loss, algorithms,  pdf,

                                 title, title1, title2,title3, title4, title5,title6, title7, title8,title9, title10, title11,title12, title13, title14,title15, title16, title17):

    font_size = 10
    bins = 120
    P = 1
    L = 1
    A = len(algorithms)

    # Manual colors
    edge_unknown = colors.to_rgba('indianred', 1)
    fill_unknown = colors.to_rgba('firebrick', 0.04)
    edge_known = colors.to_rgba('tab:blue', 1)
    fill_known = colors.to_rgba('tab:blue', 0.04)
    edge_negative = colors.to_rgba('tab:green', 1)
    fill_negative = colors.to_rgba('tab:green', 0.04)

    #define figure size
    fig  = pyplot.figure(figsize = (28,12))
    gs = fig.add_gridspec(3,6,hspace = 0.2, wspace = 0.9)
    ax = gs.subplots(sharex = True, sharey = 'row')
    ax = ax.flat

    #plot each histogram
    if scores[protocols][loss][algorithms] is not None:
      histograms = get_histogram(scores[protocols][loss][algorithms], ground_truths[protocols], bins = bins)
      ax[0].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[0].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[0].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[0].set_title(title,size = font_size)


    if scores1[protocols1][loss][algorithms] is not None:
      histograms = get_histogram(scores1[protocols1][loss][algorithms], ground_truths1[protocols1], bins = bins)
      ax[1].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[1].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[1].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[1].set_title(title1,size = font_size)


    if scores2[protocols2][loss][algorithms] is not None:
      histograms = get_histogram(scores2[protocols2][loss][algorithms], ground_truths2[protocols2], bins = bins)
      ax[2].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[2].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[2].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[2].set_title(title2,size = font_size)


    if scores3[protocols3][loss][algorithms] is not None:
      histograms = get_histogram(scores3[protocols3][loss][algorithms], ground_truths3[protocols3], bins = bins)
      ax[3].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[3].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[3].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[3].set_title(title3,size = font_size)


    if scores4[protocols4][loss][algorithms] is not None:
      histograms = get_histogram(scores4[protocols4][loss][algorithms], ground_truths4[protocols4], bins = bins)
      ax[4].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[4].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[4].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[4].set_title(title4,size = font_size)


    if scores5[protocols5][loss][algorithms] is not None:
      histograms = get_histogram(scores5[protocols5][loss][algorithms], ground_truths5[protocols5], bins = bins)
      ax[5].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[5].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[5].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[5].set_title(title5,size = font_size)


    if scores6[protocols6][loss][algorithms] is not None:
      histograms = get_histogram(scores6[protocols6][loss][algorithms], ground_truths6[protocols6], bins = bins)
      ax[6].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[6].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[6].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[6].set_title(title6,size = font_size)



    if scores7[protocols7][loss][algorithms] is not None:
      histograms = get_histogram(scores7[protocols7][loss][algorithms], ground_truths7[protocols7], bins = bins)
      ax[7].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[7].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[7].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[7].set_title(title7,size = font_size)


    if scores8[protocols8][loss][algorithms] is not None:
      histograms = get_histogram(scores8[protocols8][loss][algorithms], ground_truths8[protocols8], bins = bins)
      ax[8].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[8].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[8].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[8].set_title(title8,size = font_size)

    if scores9[protocols9][loss][algorithms] is not None:
      histograms = get_histogram(scores9[protocols9][loss][algorithms], ground_truths9[protocols9], bins = bins)
      ax[9].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[9].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[9].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[9].set_title(title9,size = font_size)

    if scores10[protocols10][loss][algorithms] is not None:
      histograms = get_histogram(scores10[protocols10][loss][algorithms], ground_truths10[protocols10], bins = bins)
      ax[10].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[10].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[10].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[10].set_title(title10,size = font_size)

    if scores11[protocols11][loss][algorithms] is not None:
      histograms = get_histogram(scores11[protocols11][loss][algorithms], ground_truths11[protocols11], bins = bins)
      ax[11].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[11].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[11].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[11].set_title(title11,size = font_size)

    if scores12[protocols12][loss][algorithms] is not None:
      histograms = get_histogram(scores12[protocols12][loss][algorithms], ground_truths12[protocols12], bins = bins)
      ax[12].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[12].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[12].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[12].set_title(title12,size = font_size)

    if scores13[protocols13][loss][algorithms] is not None:
      histograms = get_histogram(scores13[protocols13][loss][algorithms], ground_truths13[protocols13], bins = bins)
      ax[13].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[13].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[13].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[13].set_title(title13,size = font_size)

    if scores14[protocols14][loss][algorithms] is not None:
      histograms = get_histogram(scores14[protocols14][loss][algorithms], ground_truths14[protocols14], bins = bins)
      ax[14].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[14].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[14].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[14].set_title(title14,size = font_size)

    if scores15[protocols15][loss][algorithms] is not None:
      histograms = get_histogram(scores15[protocols15][loss][algorithms], ground_truths15[protocols15], bins = bins)
      ax[15].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[15].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[15].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[15].set_title(title15,size = font_size)

    if scores16[protocols16][loss][algorithms] is not None:
      histograms = get_histogram(scores16[protocols16][loss][algorithms], ground_truths16[protocols16], bins = bins)
      ax[16].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[16].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[16].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[16].set_title(title16,size = font_size)

    if scores17[protocols17][loss][algorithms] is not None:
      histograms = get_histogram(scores17[protocols17][loss][algorithms], ground_truths17[protocols17], bins = bins)
      ax[17].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[17].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[17].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[17].set_title(title17,size = font_size)




    from matplotlib.lines import Line2D
    legend_elements = [Line2D([None], [None], color=edge_known),
                        Line2D([None], [None], color=edge_negative),
                        Line2D([None], [None], color=edge_unknown)]
    legend_labels = ["Known", "Negative", "Unknown"]
    fig.legend(handles=legend_elements, labels=legend_labels,  ncol=3, loc = "lower center", bbox_to_anchor = (0.51,0.04))

    return fig

In [ ]:

#indicate if you use current or best
use_best = False
#indicate algorithm
algorithm = 'threshold'
protocol =protocol1 = protocol2 =protocol3 = protocol4 = protocol5= protocol6 = protocol7 = protocol8 =  '2'
#indicate which negative sample method you use.
method = 'noise'
#indicate loss function
loss ='entropic'

benchmark_score = '-'
benchmark_ground_truth = ground_truths1 = ground_truths = ground_truths2 = 0
scores = '-'
scores1 = '-'
scores2 = '-'


#column 1
protocol = '1'
protocol6 = '2'
protocol12 ='3'
scores, ground_truths = load_scores(use_best, algorithm, loss, protocol ,method, 'test','benchmark_p1')
scores6, ground_truths6 = load_scores(use_best, algorithm, loss, protocol6 ,method, 'test','benchmark_p2')
scores12, ground_truths12 = load_scores(use_best, algorithm, loss, protocol12 ,method, 'test','benchmark_p3')
title = 'P1 Benchmark'
title6 = 'P2 Benchmark'
title12 = 'P3 Benchmark'

#column 3
protocol1 = '1'
protocol7 = '2'
protocol13 ='3'
scores1, ground_truths1 = load_scores(use_best, algorithm, loss, protocol1 ,method, 'test','p1')
scores7, ground_truths7= load_scores(use_best, algorithm, loss, protocol7 ,method, 'test','p2')
scores13, ground_truths13 = load_scores(use_best, algorithm, loss, protocol13 ,method, 'test','p3')
title1 = 'P1'
title7 = 'P2'
title13 = 'P3'


#column 3
protocol2 = '1'
protocol8 = '2'
protocol14 ='3'
scores2, ground_truths2 = load_scores(use_best, algorithm, loss, protocol2 ,method, 'test','p1')
scores8, ground_truths8 = load_scores(use_best, algorithm, loss, protocol8 ,method, 'test','p2')
scores14, ground_truths14 = load_scores(use_best, algorithm, loss, protocol14 ,method, 'test','p3')
title2 = 'P1'
title8 = 'P2'
title14 = 'P3'


#column 4
protocol3 = '1'
protocol9 = '2'
protocol15 ='3'
scores3, ground_truths3 = load_scores(use_best, algorithm, loss, protocol3 ,method, 'test', 'p1')
scores9, ground_truths9 = load_scores(use_best, algorithm, loss, protocol9 ,method, 'test', 'p2')
scores15, ground_truths15 = load_scores(use_best, algorithm, loss, protocol15 ,method, 'test', 'p3')
title3 = 'P1'
title9 = 'P2'
title15 = 'P3'


#column 5
protocol4 = '1'
protocol10 = '2'
protocol16 ='3'
scores4, ground_truths4 = load_scores(use_best, algorithm, loss, protocol4 ,method, 'test','p1')
scores10, ground_truths10 = load_scores(use_best, algorithm, loss, protocol10,method, 'test','p2')
scores16, ground_truths16 = load_scores(use_best, algorithm, loss, protocol16 ,method, 'test','p3')
title4 = 'P1'
title10 ='P2'
title16 = 'P3'


#column 6
protocol5 = '1'
protocol11 = '2'
protocol17 ='3'
scores5, ground_truths5 = load_scores(use_best, algorithm, loss, protocol5 ,method, 'test','p1')
scores11, ground_truths11 = load_scores(use_best, algorithm, loss, protocol11,method, 'test','p2')
scores17, ground_truths17 = load_scores(use_best, algorithm, loss, protocol17,method, 'test','p3')
title5 = 'P1'
title11 ='P2'
title17 = 'P3'




#name output file
name = 'Comparison_scoreDistribution'
output_file_directory = f"{name}.pdf"


pdf = PdfPages(output_file_directory)

#plot figure
fig1 = plot_score_distributions6( scores,ground_truths,
                                scores1, ground_truths1 ,
                                 scores2, ground_truths2,
                                 scores3, ground_truths3,
                                 scores4, ground_truths4,
                                 scores5, ground_truths5,
                                 scores6, ground_truths6,
                                 scores7, ground_truths7,
                                 scores8, ground_truths8,
                                 scores9, ground_truths9,
                                 scores10, ground_truths10,
                                 scores11, ground_truths11,
                                 scores12, ground_truths12,
                                 scores13, ground_truths13,
                                 scores14, ground_truths14,
                                scores15, ground_truths15,
                                 scores16, ground_truths16,
                                 scores17, ground_truths17,



                                 protocol,protocol1,protocol2, protocol3, protocol4, protocol5, protocol6, protocol7, protocol8,protocol9, protocol10, protocol11, protocol12, protocol13, protocol14, protocol15, protocol16, protocol17,
                                 loss, algorithm,  pdf,

                                 title, title1, title2,title3, title4, title5,title6, title7, title8,title9, title10, title11,title12, title13, title14,title15, title16, title17)

#show figure and save it under the given name
pyplot.show()
pdf.savefig(fig1, bbox_inches = 'tight')
pdf.close()

In [ ]:
#version for 2 methods
def plot_score_distributions_2( scores, ground_truths,
                               scores1, ground_truths1,
                                scores2, ground_truths2,
                                scores3, ground_truths3,
                                scores4, ground_truths4,
                                scores5, ground_truths5 ,

                                protocols,protocols1,protocols2,protocols3,protocols4,protocols5,
                                loss, algorithms,  pdf,
                                title, title1, title2, title3, title4, title5):

    font_size = 10
    bins = 120
    P = 1
    L = 1
    A = len(algorithms)

    # Manual colors
    edge_unknown = colors.to_rgba('indianred', 1)
    fill_unknown = colors.to_rgba('firebrick', 0.04)
    edge_known = colors.to_rgba('tab:blue', 1)
    fill_known = colors.to_rgba('tab:blue', 0.04)
    edge_negative = colors.to_rgba('tab:green', 1)
    fill_negative = colors.to_rgba('tab:green', 0.04)


    fig  = pyplot.figure(figsize = (12,8))
    gs = fig.add_gridspec(3,2,hspace = 0.2, wspace = 0.08)
    ax = gs.subplots(sharex = True, sharey = 'row')
    ax = ax.flat

    #plot the different histograms
    if scores[protocols][loss][algorithms] is not None:
      histograms = get_histogram(scores[protocols][loss][algorithms], ground_truths[protocols], bins = bins)
      ax[0].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[0].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[0].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[0].set_title(title,size = font_size)

    if scores1[protocols1][loss][algorithms] is not None:
      histograms = get_histogram(scores1[protocols1][loss][algorithms], ground_truths1[protocols1], bins = bins)
      ax[1].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[1].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[1].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[1].set_title(title1,size = font_size)


    if scores2[protocols2][loss][algorithms] is not None:
      histograms = get_histogram(scores2[protocols2][loss][algorithms], ground_truths2[protocols2], bins = bins)
      ax[2].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[2].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[2].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[2].set_title(title2,size = font_size)


    if scores3[protocols3][loss][algorithms] is not None:
      histograms = get_histogram(scores3[protocols3][loss][algorithms], ground_truths3[protocols3], bins = bins)
      ax[3].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[3].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[3].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[3].set_title(title3,size = font_size)


    if scores4[protocols4][loss][algorithms] is not None:
      histograms = get_histogram(scores4[protocols4][loss][algorithms], ground_truths4[protocols4], bins = bins)
      ax[4].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[4].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[4].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[4].set_title(title4,size = font_size)


    if scores5[protocols5][loss][algorithms] is not None:
      histograms = get_histogram(scores5[protocols5][loss][algorithms], ground_truths5[protocols5], bins = bins)
      ax[5].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[5].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[5].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[5].set_title(title5,size = font_size)

    from matplotlib.lines import Line2D
    legend_elements = [Line2D([None], [None], color=edge_known),
                        Line2D([None], [None], color=edge_negative),
                        Line2D([None], [None], color=edge_unknown)]
    legend_labels = ["Known", "Negative", "Unknown"]
    fig.legend(handles=legend_elements, labels=legend_labels,  ncol=3, loc = "lower center", bbox_to_anchor = (0.52,-0.0))

    return fig

In [ ]:
use_best = False
algorithm = 'threshold'
protocol =protocol1 = protocol2 =protocol3 = protocol4 = protocol5= protocol6 = protocol7 = protocol8 =  '2'
method = 'noise'
loss ='entropic'

scores = '-'
scores1 = '-'
scores2 = '-'

title1 = 'P1'
title3 = 'P2'
title5 = 'P3 '

#column 1
protocol = '1'
protocol2 = '2'
protocol4 ='3'
scores, ground_truths = load_scores(use_best, algorithm, loss, protocol ,method, 'test','p1')
scores2, ground_truths2 = load_scores(use_best, algorithm, loss, protocol2 ,method, 'test','p2')
scores4, ground_truths4 = load_scores(use_best, algorithm, loss, protocol4 ,method, 'test','p3')
title = 'P1'
title2 = 'P2'
title4 = 'P3'

#column 2
protocol1 = '1'
protocol3 = '2'
protocol5 ='3'
scores1, ground_truths1 = load_scores(use_best, algorithm, loss, protocol1 ,method, 'test', 'p1')
scores3, ground_truths3 = load_scores(use_best, algorithm, loss, protocol3 ,method, 'test', 'p2')
scores5, ground_truths5 = load_scores(use_best, algorithm, loss, protocol5 ,method, 'test', 'p3')
title1 ='P1'
title3 = 'P2'
title5 = 'P3'


name = 'Comparison_ScoreDistribution'

output_file_directory = f"{name}.pdf"


pdf = PdfPages(output_file_directory)

fig1 = plot_score_distributions_2( scores,ground_truths,scores1, ground_truths1 ,scores2, ground_truths2, scores3, ground_truths3,scores4, ground_truths4,scores5, ground_truths5,  protocol,protocol1,protocol2, protocol3, protocol4, protocol5, loss, algorithm,  pdf, title, title1, title2,title3, title4, title5)

pyplot.show()
pdf.savefig(fig1, bbox_inches = 'tight')
pdf.close()


In [ ]:
#Version for 3 methods 
def plot_score_distributions_3( scores, ground_truths,
                               scores1, ground_truths1,
                                scores2, ground_truths2,
                                scores3, ground_truths3,
                                scores4, ground_truths4,
                                scores5, ground_truths5 ,
                                scores6, ground_truths6,
                                scores7, ground_truths7,
                                scores8, ground_truths8 ,

                                protocols,protocols1,protocols2,protocols3,protocols4,protocols5,protocols6,protocols7,protocols8,
                                loss, algorithms,  pdf,
                                title, title1, title2, title3, title4, title5,title6, title7, title8):

    font_size = 10
    bins = 120
    P = 1
    L = 1
    A = len(algorithms)

    # Manual colors
    edge_unknown = colors.to_rgba('indianred', 1)
    fill_unknown = colors.to_rgba('firebrick', 0.04)
    edge_known = colors.to_rgba('tab:blue', 1)
    fill_known = colors.to_rgba('tab:blue', 0.04)
    edge_negative = colors.to_rgba('tab:green', 1)
    fill_negative = colors.to_rgba('tab:green', 0.04)


    fig  = pyplot.figure(figsize = (12,8))
    gs = fig.add_gridspec(3,3,hspace = 0.2, wspace = 0.08)
    ax = gs.subplots(sharex = True, sharey = 'row')
    ax = ax.flat

    if scores[protocols][loss][algorithms] is not None:
      histograms = get_histogram(scores[protocols][loss][algorithms], ground_truths[protocols], bins = bins)
      ax[0].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[0].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[0].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[0].set_title(title,size = font_size)

    if scores1[protocols1][loss][algorithms] is not None:
      histograms = get_histogram(scores1[protocols1][loss][algorithms], ground_truths1[protocols1], bins = bins)
      ax[1].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[1].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[1].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[1].set_title(title1,size = font_size)


    if scores2[protocols2][loss][algorithms] is not None:
      histograms = get_histogram(scores2[protocols2][loss][algorithms], ground_truths2[protocols2], bins = bins)
      ax[2].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[2].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[2].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[2].set_title(title2,size = font_size)


    if scores3[protocols3][loss][algorithms] is not None:
      histograms = get_histogram(scores3[protocols3][loss][algorithms], ground_truths3[protocols3], bins = bins)
      ax[3].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[3].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[3].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[3].set_title(title3,size = font_size)


    if scores4[protocols4][loss][algorithms] is not None:
      histograms = get_histogram(scores4[protocols4][loss][algorithms], ground_truths4[protocols4], bins = bins)
      ax[4].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[4].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[4].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[4].set_title(title4,size = font_size)

    if scores5[protocols5][loss][algorithms] is not None:
      histograms = get_histogram(scores5[protocols5][loss][algorithms], ground_truths5[protocols5], bins = bins)
      ax[5].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[5].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[5].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[5].set_title(title5,size = font_size)

    if scores6[protocols6][loss][algorithms] is not None:
      histograms = get_histogram(scores6[protocols6][loss][algorithms], ground_truths6[protocols6], bins = bins)
      ax[6].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[6].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[6].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[6].set_title(title6,size = font_size)

    if scores7[protocols7][loss][algorithms] is not None:
      histograms = get_histogram(scores7[protocols7][loss][algorithms], ground_truths7[protocols7], bins = bins)
      ax[7].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[7].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[7].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[7].set_title(title7,size = font_size)

    if scores8[protocols8][loss][algorithms] is not None:
      histograms = get_histogram(scores8[protocols8][loss][algorithms], ground_truths8[protocols8], bins = bins)
      ax[8].stairs(histograms["known"][0], histograms["known"][1], fill=True, color=fill_known, edgecolor=edge_known, linewidth=1)
      ax[8].stairs(histograms["unknown"][0], histograms["unknown"][1], fill=True, color=fill_unknown, edgecolor=edge_unknown, linewidth=1)
      ax[8].stairs(histograms["negative"][0], histograms["negative"][1], fill=True, color=fill_negative, edgecolor=edge_negative, linewidth=1)

    ax[8].set_title(title8,size = font_size)

    from matplotlib.lines import Line2D
    legend_elements = [Line2D([None], [None], color=edge_known),
                        Line2D([None], [None], color=edge_negative),
                        Line2D([None], [None], color=edge_unknown)]
    legend_labels = ["Known", "Negative", "Unknown"]
    fig.legend(handles=legend_elements, labels=legend_labels,  ncol=3, loc = "lower center", bbox_to_anchor = (0.52,-0.0))

    return fig

In [ ]:
use_best = False
algorithm = 'threshold'
protocol =protocol1 = protocol2 =protocol3 = protocol4 = protocol5= protocol6 = protocol7 = protocol8 =  '2'
method = 'noise'
loss ='entropic'

benchmark_score = '-'
benchmark_ground_truth = ground_truths1 = ground_truths = ground_truths2 = 0
scores = '-'
scores1 = '-'
scores2 = '-'

#column 1
protocol = '1'
protocol3 = '2'
protocol6 ='3'
scores, ground_truths = load_scores(use_best, algorithm, loss, protocol ,method, 'test','p1')
scores3, ground_truths3 = load_scores(use_best, algorithm, loss, protocol3 ,method, 'test','p2')
scores6, ground_truths6 = load_scores(use_best, algorithm, loss, protocol6 ,method, 'test','p3')
title = 'P1 '
title3 = 'P2'
title6 = 'P3'

#column 2
protocol1 = '1'
protocol4 = '2'
protocol7 ='3'
scores1, ground_truths1 = load_scores(use_best, algorithm, loss, protocol1 ,method, 'test', 'p1')
scores4, ground_truths4 = load_scores(use_best, algorithm, loss, protocol4 ,method, 'test', 'p2')
scores7, ground_truths7 = load_scores(use_best, algorithm, loss, protocol7 ,method, 'test', 'p3')
title1 ='P1 '
title4 = 'P2 '
title7 = 'P3 '

#column 3
protocol2 = '1'
protocol5 = '2'
protocol8 ='3'
scores2, ground_truths2 = load_scores(use_best, algorithm, loss, protocol2 ,method, 'test', 'p1')
scores5, ground_truths5 = load_scores(use_best, algorithm, loss, protocol5 ,method, 'test', 'p2')
scores8, ground_truths8 = load_scores(use_best, algorithm, loss, protocol8 ,method, 'test', 'p3')
title2 ='P1'
title5 = 'P2'
title8 = 'P3 '

#define output file
name = 'Comparison_ScoreDistribution'
output_file_directory = f"{name}.pdf"


pdf = PdfPages(output_file_directory)
fig1 = plot_score_distributions_3( scores, ground_truths,
                               scores1, ground_truths1,
                                scores2, ground_truths2,
                                scores3, ground_truths3,
                                scores4, ground_truths4,
                                scores5, ground_truths5 ,
                                scores6, ground_truths6,
                                scores7, ground_truths7,
                                scores8, ground_truths8 ,

                                protocol,protocol1,protocol2,protocol3,protocol4,protocol5,protocol6,protocol7,protocol8,
                                loss, algorithm,  pdf,
                                title, title1, title2, title3, title4, title5,title6, title7, title8)


pyplot.show()
pdf.savefig(fig1, bbox_inches = 'tight')
pdf.close()